In [3]:
import csv

from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import SystemMessage, HumanMessagePromptTemplate

In [5]:
def load_cases(file):
    with open(file, 'r') as f:
        reader = csv.DictReader(f)
        for case in reader:
            yield case

# Load the cases actually
cases = [case for case in load_cases('../data/cases.csv')]
print(f'Loaded {len(cases)} cases')

Loaded 34 cases


In [6]:
def query_create_variants(case, dimensions, fewshot_examples = []):
    template = ChatPromptTemplate.from_messages([
        ("system", "You are an AI bot that writes scenarios that legal experts can help answer. You will be provided 3 pieces of information from the user. The first is an initial scenario. The second is a key dimension of that scenario. The third is the number of scenarios to generate. You will rewrite the scenario provided to you by varying it along the key dimension. To do so, consider n different possibilities along the key dimension, where n is the number of scenarios to generate. Your response should be n scenarios, each one based off the original but edited to include a different possibility of the key dimension. Preserve the point-of-view written in the original scenario (for example, if it was written in first-person, keep the rewritten scenarios in first-person)."),
        ("human", "{case}"),
        ("ai", "Ok, I now have the scenario. What is the key dimension to consider when rewriting the scenario?"),
        ("human", "{dimension}"),
        ("ai", "Ok, I now have the scenario and the key dimension. How many possibilities of that dimension should I use to rewrite the scenario?"),
        ("human", "{n}")
    ])
    messages = template.format_messages(
        case="My mother and I are board members of an advertising company incorporated in Delaware. We have business trips every now and then; for example, to meet with potential advertisers. Is it okay to file trip expenses as deductible expenses of the company? What are the criteria to determine whether it is appropriate to use company assets that are not being charged for embezzlement in this family corporation setting?",
        dimension="Would they have gone otherwise?",
        n=3
    )
    return chat(messages).content